# OpenRouter API Demo Notebook

This notebook demonstrates how to use OpenRouter API with both direct API calls and OpenAI client.

## What is OpenRouter?

OpenRouter is a unified API platform that acts as a marketplace for large language models (LLMs). It aggregates hundreds of AI models from different providers into a single, standardized API, allowing you to access models from Google, Anthropic, Meta, and others through one key and one interface.

## Is it Free?

**Yes, OpenRouter offers a free tier with limitations:**
- Many models are available for free (marked with `:free` tag)
- Free models may be slower with higher latency
- Rate limits apply (check current limits in OpenRouter docs)
- Paid models require adding credits to your account

## Major Models Available

| Provider | Example Models |
|----------|----------------|
| Google | `google/gemini-2.0-flash-exp:free`, `google/gemini-2.5-pro-preview` |
| Anthropic | `anthropic/claude-3.5-sonnet` |
| Meta | `meta-llama/llama-3.1-70b-instruct:free` |
| DeepSeek | `deepseek/deepseek-chat-v3-0324:free` |
| Mistral | Various Mistral models |
| OpenAI | `openai/gpt-3.5-turbo` |

## Cell 1: Using OpenRouter API Directly

This method uses the `requests` library to call OpenRouter API directly.

In [ ]:
import requests
import json
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set your API key and the endpoint
API_KEY = os.getenv("OPENROUTER_API_KEY")
url = "https://openrouter.ai/api/v1/chat/completions"

# Choose a model. Using a free model here.
model = "openai/gpt-4o"

# Define the request headers
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
}

# Define the prompt and message history
messages = [
    {"role": "user", "content": "Explain what OpenRouter is in 2-3 sentences."}
]

# Create the request data
data = {
    "model": model,
    "messages": messages,
    "max_tokens": 500
}

print("Calling OpenRouter API directly...")
# Make the API call
response = requests.post(url=url, headers=headers, json=data)

# Check if the request was successful and print the result
if response.status_code == 200:
    result = response.json()
    assistant_message = result['choices'][0]['message']['content']
    print("✅ Success! Response:")
    print("-" * 50)
    print(assistant_message)
    print("-" * 50)
    
    # Print token usage
    if 'usage' in result:
        usage = result['usage']
        print(f"\nToken usage: {usage.get('prompt_tokens', 'N/A')} prompt, {usage.get('completion_tokens', 'N/A')} completion")
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

## Cell 2: Using OpenAI Client with OpenRouter

This method uses the official `openai` Python package, which is compatible with OpenRouter by changing the `base_url`.

In [ ]:

import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # Loads the .env file

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key= os.environ["OPENROUTER_API_KEY"],
)

# Use the standard chat completions endpoint
response = client.chat.completions.create(
    model="openai/gpt-4o",  # Using a standard mini model name
    messages=[
        {"role": "user", "content": "2+2=?"}
    ],
    max_tokens=5
)

# Access the content from the response object
print(response.choices[0].message.content)

## Cell 3: Compare Different Models

Let's test the same prompt with different models available on OpenRouter.

In [16]:
def test_openrouter_model(model_name, prompt):
    """Test a specific model with OpenRouter"""
    try:
        client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=os.getenv("OPENROUTER_API_KEY"),
        )
        
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=300
        )
        
        return {
            "success": True,
            "content": response.choices[0].message.content,
            "model": response.model,
            "tokens": f"{response.usage.prompt_tokens} + {response.usage.completion_tokens}"
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

# Test different models
test_prompt = "In one sentence, what is artificial intelligence?"
models_to_test = [
    "kwaipilot/kat-coder-pro:free",
    "nousresearch/hermes-3-llama-3.1-405b:free",
    "google/gemini-2.0-flash-exp:free"
]

print("Testing different models with the same prompt...")
print("Prompt:", test_prompt)
print("=" * 60)

for model in models_to_test:
    print(f"\n🧠 Testing: {model}")
    result = test_openrouter_model(model, test_prompt)
    
    if result["success"]:
        print(f"✅ Response: {result['content']}")
        print(f"📊 Tokens: {result['tokens']}")
    else:
        print(f"❌ Error: {result['error']}")
    print("-" * 40)

Testing different models with the same prompt...
Prompt: In one sentence, what is artificial intelligence?

🧠 Testing: kwaipilot/kat-coder-pro:free
✅ Response: Artificial intelligence is the simulation of human intelligence processes by machines, especially computer systems, enabling them to perform tasks such as learning, reasoning, problem-solving, and decision-making.
📊 Tokens: 17 + 36
----------------------------------------

🧠 Testing: nousresearch/hermes-3-llama-3.1-405b:free
✅ Response: Artificial intelligence is the development of computer systems that are able to perform tasks that normally require human intelligence, such as visual perception, speech recognition, decision-making, and language translation.
📊 Tokens: 691 + 36
----------------------------------------

🧠 Testing: google/gemini-2.0-flash-exp:free
❌ Error: Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'meta

## Important Notes

1. **API Key**: Replace `your_actual_openrouter_api_key_here` with your actual OpenRouter API key
2. **Free Models**: Look for models with `:free` suffix for no-cost usage
3. **Rate Limits**: Be aware of OpenRouter's rate limits for free usage
4. **Model Availability**: Check [OpenRouter Models](https://openrouter.ai/models) for the latest available models

## Next Steps

- Visit [OpenRouter Docs](https://openrouter.ai/docs) for detailed API documentation
- Explore different models and their capabilities
- Monitor your usage in the OpenRouter dashboard